In [1]:
import os
import json
import time
import logging
import base64
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta, timezone
from PIL import Image
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20.exceptions import V20Error
import openai
import tiktoken

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M15'  # 15 minutes granularity
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '15 minutes'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    rec = {
                        'time': candle.get('time')[0:19],
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def calculate_fibonacci_retracement(data):
    max_price = data['high'].max()
    min_price = data['low'].min()
    diff = max_price - min_price
    levels = [max_price - diff * r for r in [0.236, 0.382, 0.5, 0.618, 0.786]]
    return levels

def plot_candlestick_chart(df, filename):
    df.index = pd.to_datetime(df['time'])
    df.index.name = 'Date'
    df['SMA20'] = df['close'].rolling(window=20).mean()
    df['SMA50'] = df['close'].rolling(window=50).mean()
    df['RSI'] = calculate_rsi(df['close'])

    # Correct calculation for support and resistance
    support = df['low'].rolling(window=20).min()
    resistance = df['high'].rolling(window=20).max()

    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--')
    
    addplots = [
        mpf.make_addplot(df['SMA20'], color='blue'),
        mpf.make_addplot(df['SMA50'], color='orange'),
        mpf.make_addplot(df['RSI'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(support, color='green', linestyle='dashed'),
        mpf.make_addplot(resistance, color='red', linestyle='dashed'),
    ]
    
    kwargs = dict(type='candle', style=s, addplot=addplots, volume=True, figscale=1.0, figratio=(10, 10), title=pair, ylabel='Price')
    mpf.plot(df, **kwargs, savefig=dict(fname=filename, dpi=100, pad_inches=0.1))

def compress_image(input_path, output_path, quality=85):
    with Image.open(input_path) as img:
        img = img.convert('RGB')  # Convert to RGB
        img = img.resize((510, 510), Image.LANCZOS)  # Resize to 510x510
        img.save(output_path, 'JPEG', quality=quality)  # Save with reduced quality

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def save_prompt_to_txt(prompt_content, filename):
    with open(filename, 'w') as file:
        json.dump(prompt_content, file, indent=4)

def analyze_data_with_gpt4o(indicators_summary, image_base64):
    logging.info("Sending data to OpenAI API for analysis")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": "Analyze the indicators data for EUR/USD and provide order details with profit/loss greater than 2. Combine all patterns found and aggregate them to find the best pattern with the highest probability, most profit/loss, and strongest win signs. The patterns should not be in the past and should happen now or in the near future. Dont send any descriotions or extra text.\n\n"
                   "Provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"orders\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"# minutes\",\n"
                   "            \"pattern_name\": \"Pattern Name\",\n"
                   "            \"confidence_percentage\": ##,\n"
                   "            \"action\": \"####\",\n"
                   "            \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"profit_loss_ratio\": ##,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "        }\n"
                   "    ],\n"
                   "    \"best_pattern\": {\n"
                   "        \"pattern_name\": \"Pattern Name\",\n"
                   "        \"confidence_percentage\": ##,\n"
                   "        \"action\": \"####\",\n"
                   "        \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"profit_loss_ratio\": ##,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "    }\n"
                   "}\n",
        "image": f"data:image/png;base64,{image_base64}",
        "indicators_summary": indicators_summary
    }

    # Calculate tokens before adding image
    enc = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(enc.encode(json.dumps(prompt_content)))
    data_cost = num_tokens * 0.000005
    image_cost = 0.001275  # Fixed image cost based on 512x512 px
    
    logging.info(f"Data tokens before adding image: {num_tokens}, Estimated Data Cost: ${data_cost:.6f}")

    save_prompt_to_txt(prompt_content, 'final_prompt.txt')
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data, data_cost, image_cost
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None, data_cost, image_cost

def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    orders = analysis.get("orders", [])
    best_pattern = analysis.get("best_pattern", {})
    
    for order in orders:
        if order.get("profit_loss_ratio", 0) > 2:
            logging.info(f"Order Details - Action: {order['action']}, Entry Price: {order['entry_price']}, Take Profit: {order['take_profit']}, Stop Loss: {order['stop_loss']}")
            
            order_details = {
                'action': order['action'],
                'entry_price': order['entry_price'],
                'take_profit': order['take_profit'],
                'stop_loss': order['stop_loss'],
                'deadline_date': order['deadline_date']
            }
            response = place_order(order_details)
            
            if 'orderCancelTransaction' in response:
                logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
            return order_details

    logging.info(f"Best Pattern Details - Action: {best_pattern['action']}, Entry Price: {best_pattern['entry_price']}, Take Profit: {best_pattern['take_profit']}, Stop Loss: {best_pattern['stop_loss']}")
    return best_pattern

def place_order(order_details):
    instrument = "EUR_USD"
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

def save_data_to_csv(data, filename):
    data.to_csv(filename, index=False)

# Main execution
while True:
    start_time = (datetime.now(timezone.utc) - timedelta(days=4)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    # Calculate indicators
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    prices['Fibonacci'] = pd.Series(calculate_fibonacci_retracement(prices))

    save_data_to_csv(prices, 'forex_data_and_indicators.csv')

    # Collect all the data points
    indicators_summary = {
        "ClosePrices": np.round(prices['close'], 4).to_list(),
        "Volume": prices['volume'].to_list(),
        "RSI": np.round(prices['RSI'], 2).to_list(),
        "MACD": np.round(prices['MACD'], 4).to_list(),
        "MACD_Signal": np.round(prices['MACD_Signal'], 4).to_list(),
        "Bollinger_Bands": {
            "Middle": np.round(prices['Bollinger_Mid'], 4).to_list(),
            "Upper": np.round(prices['Bollinger_Upper'], 4).to_list(),
            "Lower": np.round(prices['Bollinger_Lower'], 4).to_list()
        },
        "Fibonacci": np.round(prices['Fibonacci'], 4).to_list()
    }

    # Plot and encode the image
    filename = "normal_chart.png"
    compressed_filename = "compressed_chart.jpg"
    plot_candlestick_chart(prices, filename)  # Use color image
    compress_image(filename, compressed_filename, quality=85)  # Compress without grayscale
    image_base64 = encode_image(compressed_filename)
    image_size_kb = len(base64.b64decode(image_base64)) / 1024

    analysis_result, data_cost, image_cost = analyze_data_with_gpt4o(indicators_summary, image_base64)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    if analysis_result:
        response_tokens = analysis_result["usage"]["completion_tokens"]
        response_cost = response_tokens * 0.000015  # Adjust based on actual token cost
        logging.info(f"Response received: {response_tokens} tokens, Cost: ${response_cost:.6f}")

        prompt_tokens = analysis_result["usage"]["prompt_tokens"]
        prompt_cost = prompt_tokens * 0.00005  # Adjust based on actual token cost
        logging.info(f"Prompt sent: {prompt_tokens} tokens, Cost: ${prompt_cost:.6f}")

        order_details = extract_and_place_order(analysis_result)
    
        if order_details:
            logging.info("Order placed based on analysis.")
    
    total_cost = data_cost + response_cost + image_cost
    logging.info(f"Total cost for this run: Data: ${data_cost:.6f}, Image: ${image_cost:.6f}, Response: ${response_cost:.6f}, Total: ${total_cost:.6f}")

    logging.info("Waiting for 30 minutes before next run...")
    time.sleep(1800)


INFO:root:Fetching forex data from 2024-06-05T22:04:49Z to 2024-06-09T22:04:49Z with granularity M15 for instrument EUR_USD
INFO:root:Sending data to OpenAI API for analysis
INFO:root:Data tokens before adding image: 58134, Estimated Data Cost: $0.290670
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9YKweqs88FzxpWlrtRBiZikvPHEw9",
    "object": "chat.completion",
    "created": 1717970692,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"orders\": [\n        {\n            \"timeframe\": \"60 minutes\",\n            \"pattern_name\": \"Bullish RSI Divergence\",\n            \"confidence_percentage\": 85,\n            \"action\": \"BUY\",\n            \"entry_price\": 1.0869,\n            \"take_profit\": 1.0895,\n            \"stop_loss\": 1.0840,\n            \"profit_loss_ratio\": 2.67,\n            \"deadline_date\": \"2023-10-